In [ ]:
! python3 --version

In [ ]:
! ls ../thesis/

In [1]:
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

import os
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
N = 10000
#ETD_PATH = '/mnt/ceph/cme/thesis/'
ETD_PATH = '../thesis/thesis/'
list_of_docs = os.listdir(ETD_PATH)[:100]
len(list_of_docs)

100

In [3]:
def data_generator():
    
    for idx, folder in enumerate(list_of_docs):
        for _ in os.listdir(ETD_PATH + folder):
            file_name = ''
            if _.endswith('.pdf.txt') or _.endswith('.PDF.txt'):
                file_name = _
                break
        if file_name:
            with open(ETD_PATH + folder + '/' + file_name) as f:
                document = f.read()
                
                tokens = nltk.word_tokenize(document.lower().translate(str.maketrans('', '', string.punctuation)))
                stems = []
                
                for item in tokens:
                    if item in stopwords.words():
                        continue
                    stems.append(PorterStemmer().stem(item))

                #tokens = gensim.utils.simple_preprocess(document)
                
                yield gensim.models.doc2vec.TaggedDocument(stems, [int(folder) + 1])

In [ ]:
train_corpus = list(data_generator())
len(train_corpus), type(train_corpus[0])

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x7f68951b8830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/nltk/data.py", line 1281, in __del__
    self.close()
  File "/usr/local/lib/python3.7/site-packages/nltk/data.py", line 1310, in close
    self.stream.close()
KeyboardInterrupt


In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=128, min_count=2, epochs=20)
model.build_vocab(train_corpus)

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save('../obj/doc2vec1')